# Building a convnet for ✊✋✌
This presents how to build a convnet from scratch to classify images of rock-paper-scissors.  It is meant as a teaching activity to demonstrate the following concepts in practice:
- how images are represented and handled in software
- how to prepare a machine learning dataset
- how a full machine learning pipeline looks
- data preprocessing
- data augmentation and its importance in a 
- overfitting, underfitting

We use the high-level deep learning library Keras, but the concepts are general and we don't put much focus on the specifics of the code.

## Step 0: defining the problem
What problem do we want to solve, exactly?  We want to build a piece of software that, given an image as input that represents an hand making one of the three ✊✋✌ gestures, produces as output a classification of the image in one of the three classes.

In the following, we will adopt this convention
- class 0 is ✊ rock
- class 1 is ✋ paper
- class 2 is ✌ scissors

## Step 1: building a dataset
We are starting from scratch, so we need to shoot our own dataset; the best option is that multiple students/groups shoot plenty of images in parallel and then the data is somehow collected.  Whatever the process, in the end we want to have all pics in three different directories, one per class.  Format can be either jpg or png, and landscape/portrait, aspect ratio and resolution don't matter and can be mixed.

With some attention to logistics, this can be done in about 10-30 minutes.

Guidelines for shooting images. 
- We don't need high resolution: use the lowest resolution/quality allowed by the phone (this reduces the size of the dataset and speeds up data transfer).
- The hand must be more or less in the center of the image; it should not fill the whole image, but it should not be too small either.  ![caption](figures/guidelines.jpg)
- we want the dataset to represent as much variability as possible: if we want the classifier to work for all hand orientations, try to have examples for all of them; if we want to handle many different lightling conditions, try to have some pictures for different lightings;
- avoid poses that are ambiguous, unless you want to make your job harder: e.g., don't include in the dataset images of paper or scissors taken from the side;
- avoid having two images in the dataset that are almost the same: change the camera and hand pose at least a little bit; this is important because in the following code we randomly split training and testing data.

Remember that we need the images for each class to be in its own directory. To make this simpler, it helps to shoot first all images of rock, then all images of paper, then all images of scissors, and finally sort the images by time in the file manager and group them accordingly.

Place all images in three directories named `c0/`, `c1/`, and `c2/`.  Make sure that each directory only contains image files.

## Step 2: read in images and have a look at them
Let's first import what we need

In [1]:
%matplotlib nbagg

# General imports
import matplotlib.pyplot as plt
import pathlib
import skimage
import skimage.transform
import skimage.viewer
import pandas as pd
import numpy as np
import time
import keras.utils.np_utils

# Setup to show interactive jupyter widgets
from IPython.display import Image, display
from ipywidgets import interact, fixed
import ipywidgets as widgets
def imgplotList(i,data):
    plt.figure(figsize=(10,10))
    plt.imshow(data[i],interpolation="nearest")
    plt.show()

Using TensorFlow backend.


In [2]:
# Define where datasets are located
dataset_directory = pathlib.Path(".")/"datasets"/"final split"

# Define which datasets we should consider.
# Each dataset is a directory withing dataset_directory
# and must contain three subdirectories: (c0, c1, c2) for (rock, paper, scissors).
dnames = ["D1","D2","D3","D4","D5","testing"]

# Now check the data
ddirs=[dataset_directory/dn for dn in dnames] # directories of the dataset
cdirs={}
for ddir in ddirs:
    cdirs.update({ddir/"c0":0,
                  ddir/"c1":1,
                  ddir/"c2":2})
names = ["rock", "paper", "scissors"]
for cdir,cdir_class in cdirs.items():
    assert(cdir.exists())
    print("Found directory {} containing class {}".format(cdir,names[cdir_class]))

Found directory datasets\final split\D2\c0 containing class rock
Found directory datasets\final split\D1\c1 containing class paper
Found directory datasets\final split\D4\c2 containing class scissors
Found directory datasets\final split\testing\c1 containing class paper
Found directory datasets\final split\D3\c0 containing class rock
Found directory datasets\final split\D2\c1 containing class paper
Found directory datasets\final split\D3\c2 containing class scissors
Found directory datasets\final split\D5\c0 containing class rock
Found directory datasets\final split\D5\c1 containing class paper
Found directory datasets\final split\testing\c0 containing class rock
Found directory datasets\final split\D4\c1 containing class paper
Found directory datasets\final split\D1\c2 containing class scissors
Found directory datasets\final split\D2\c2 containing class scissors
Found directory datasets\final split\D4\c0 containing class rock
Found directory datasets\final split\D1\c0 containing class

Let's try to read the first image from the first directory, and visualize it.  Note that the tool allows you to zoom in order to see the individual pixels.

This cell can be safely skipped.

In [3]:
im = skimage.io.imread(list(list(cdirs.keys())[0].glob("*"))[0])
viewer=skimage.viewer.ImageViewer(im)
viewer.show()
# Note: you have to close the window to continue

[]

We now read in all images, cut the central square (with an edge as long as the shorter dimension), and resize it to 500x500 pixels.  Whatever the initial size and orientation of the images, we will end up with a bunch of 500x500 RGB squares in uint8.  These should be small enough that unless the dataset is huge, all should fit in memory.

We make a pandas dataframe with the data, with two columns:
* image: a $500 \times 500 \times 3$ uint8 numpy array
* label: on of 0, 1 or 2
* file: the full path of the image

Note: this step may take a lot of time.

In [4]:
imagesize = 500

dataset=[]

import warnings

from tqdm import tqdm
for cdir,cn in cdirs.items():
    for f in tqdm(list(cdir.glob("*"))):
        
        im=skimage.io.imread(f)
        h,w=im.shape[0:2] # height, width
        sz=min(h,w)
        im=im[(h//2-sz//2):(h//2+sz//2),(w//2-sz//2):(w//2+sz//2),:] # defines the central square
        
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            im=skimage.img_as_ubyte(skimage.transform.resize(im,(imagesize,imagesize))) # resize it to 500x500, whatever the original resolution
        
        dataset.append({
            "file":f,
            "label":cn,
            "image":im})
        
print("Done")

ImportError: No module named 'tqdm'

We make a pandas dataframe for the dataset, and create a "dn" field containing the name of the dataset from which each image comes (as the name of the directory it was read from).

In [ ]:
dataset=pd.DataFrame(dataset)
dataset["dn"]=dataset["file"].apply(lambda x: x.parent.parts[-2])

Here are 10 random rows from that dataset

In [ ]:
dataset.sample(n=10)[["image","label","file","dn"]].style

Optionally, we can quickly scroll through the images in our dataset.

This cell can be safely skipped.

In [ ]:
viewer=skimage.viewer.CollectionViewer([r[1]["image"] for r in dataset.iterrows()])
viewer.show()

# Note: you have to close the window to continue

## Step 3: Prepare training and testing sets

How should we split training and testing data?  The code below implements a few options (run only one of the cells).

In [ ]:
# Option one (easy): use all samples from dataset named "testing"
# which contains some images taken in the same days as D1--D4, but not contained in these dirs.
te_mask = dataset["dn"]=="testing"
dataset_te=dataset[te_mask]

In [ ]:
# Option two (hard): use all samples from dataset named "D5"
te_mask = dataset["dn"]=="D5"
dataset_te=dataset[te_mask]

In [ ]:
# In any case, training samples are all other samples
dataset_tr=dataset.loc[dataset.index.difference(dataset_te.index)]

In [ ]:
# Print a summary of how many training and testing images we have sampled
import collections
pd.DataFrame(index=[0,1,2],data=collections.OrderedDict((
    ("Class name",           names),
    ("# Training images", dataset_tr["label"].value_counts()),
    ("# Testing images",  dataset_te["label"].value_counts())))).set_index("Class name")

In [ ]:
%matplotlib inline
examples=list(dataset_tr["image"])
interact(
    imgplotList, 
    i=widgets.IntSlider(min=0,max=len(examples)-1, step=1, value=0,continuous_update=False), 
    data=fixed(examples))

## Step 4: define what we feed to the Neural Network

In [ ]:
# Take image and resize to a specified size
def transform_simple(im,sz):
    imr = skimage.transform.resize(im, (sz,sz))
    return imr

# Take image and resize to a specified size, after applying data augmentation
def transform_complex(im,sz):
    if(np.random.rand()<0.5):
        im=np.fliplr(im)
    tf1 = skimage.transform.SimilarityTransform(scale = 1 / im.shape[0])
    tf2 = skimage.transform.SimilarityTransform(translation=[-0.5, -0.5])
    tf3 = skimage.transform.SimilarityTransform(rotation=np.deg2rad(np.random.uniform(0,360)))
    tf4 = skimage.transform.SimilarityTransform(scale=np.random.uniform(1,1.6))
    tf5 = skimage.transform.SimilarityTransform(translation=np.array([0.5, 0.5])+np.random.uniform(-0.1,0.1,size=2))
    tf6 = skimage.transform.SimilarityTransform(scale=sz)
    imr = skimage.transform.warp(im, (tf1+(tf2+(tf3+(tf4+(tf5+tf6))))).inverse, output_shape=(sz,sz),mode="edge")
    imr = imr*np.random.uniform(0.9,1.1,size=(1,1,3))
    imr = np.clip(imr,0,1)
    return imr

transform=transform_simple

In [ ]:
def sample(df,sz):
    r=df.sample(n=1)
    l=r["label"].iloc[0]
    im=r["image"].iloc[0]
    im=transform(im,sz)
    return im,l

def mktrte(df,N,sz):
    X = []
    y = []
    for i in range(N):
        im,l=sample(df,sz)
        X.append(im)
        y.append(l)
    X=np.array(X).astype('float32')
    y=np.array(y)
    y=keras.utils.np_utils.to_categorical(y,3)
    return X,y

def generator(df,batch_size,sz):
    while True:
        X,y = mktrte(df,batch_size,sz)
        yield (X,y)


In [ ]:
# Visualize 100 images from our input dataset
examples = list(mktrte(dataset_tr,100,patchsize)[0])
interact(imgplotList, i=widgets.IntSlider(min=0, max=len(examples)-1, step=1, value=0,continuous_update=False), data=fixed(examples))

In [ ]:
# Visualize 100 variations from our first input image (makes sense only if transform==transform_complex, i.e. if we are using data augmentation)
examples = list(mktrte(dataset_tr.iloc[[0]],100,patchsize)[0])
interact(imgplotList, i=widgets.IntSlider(min=0, max=len(examples)-1, step=1, value=0,continuous_update=False), data=fixed(examples))

## Step 5: build and train the neural network

In [ ]:
# Keras: Deep Learning library for Theano and TensorFlow
import keras
from keras.utils  import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Dropout

# NN settings
patchsize          = 64
batch_size         = 20
nb_filters         = 5     # number of convolutional filters to use
pool_size          = (2,2) # size of pooling area for max pooling
kernel_size        = (3,3) # convolution kernel size

model = Sequential()
model.add(Conv2D(nb_filters, kernel_size, padding='valid', input_shape=(patchsize,patchsize,3))) # 3 perchè rgb
model.add(Activation('relu'))
model.add(Conv2D(nb_filters, kernel_size))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = pool_size))
model.add(Conv2D(nb_filters, kernel_size))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = pool_size))
model.add(Conv2D(nb_filters, kernel_size))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = pool_size))
model.add(Flatten())
model.add(Dense(128)) # generate a fully connected layer wiht 128 outputs (arbitrary value)
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3)) # output layer
model.add(Activation('softmax')) # output activation per renderle probabilità

## compile! network
model.compile(loss='categorical_crossentropy',
              optimizer="adadelta",
              metrics=['accuracy'])
print(model.count_params())

Build a set of 1000 testing instances taken from the testing dataset.

Note: "testing" is synonym with "validation" and "evaluation"

In [ ]:
(X_test,y_test) = mktrte(dataset_te, 1000, patchsize)

# Prepare the logs directory, if it does not exist
(pathlib.Path(".")/"logs").mkdir(exist_ok=True)

While the network trains, we can monitor training loss/accuracy and testing loss/accuracy using tensorboard at http://0.0.0.0:6006

You may need to launch tensorboard first if it's not already running, by executing

`tensorboard --logdir=logs`

in a shell with the current working directory.  Check that you are within the proper conda environment, if applicable.

In [ ]:
history=model.fit_generator(
                    generator(dataset_tr, batch_size, patchsize),
                    steps_per_epoch=50, 
                    epochs=5000, 
                    verbose=1,
                    validation_data=(X_test,y_test),
                    callbacks=[keras.callbacks.TensorBoard(log_dir='./logs/'+time.strftime("%Y%m%d%H%M%S"), histogram_freq=0, write_graph=False, write_images=False)])

After training our model, we can save it to disk

In [ ]:
modelname = "myModel"

# Save model to a file
keras.models.save_model(model,"{}.model".format(modelname))

# Also save the testing dataset (may be large) so we can pick up from here later 
dataset_te.to_pickle("{}.testingdata.pickle".format(modelname))

## Step 6: Process the images of the testing set one by one
And visualize the results

In [ ]:
# Optionally, load a saved model and testing dataset
modelname = "models/model_venus"

model = keras.models.load_model("{}.model".format(modelname))
dataset_te = pd.read_pickle("{}.testingdata.pickle".format(modelname))

In [ ]:
# Show results by processing a single variation of the testing image

%matplotlib inline
def resultsShow(i, data, model):
    d = data.iloc[i]
    im = d["image"]
    l = d["label"]
    fig,axs = plt.subplots(nrows=1,ncols=3,figsize=(15,5),gridspec_kw={'width_ratios':[1,1,0.5]})
    
    imt = transform_simple(im, patchsize)
    axs[0].imshow(im)
    axs[0].set_title("Image (true class: {})".format(names[l]))
    
    axs[1].imshow(imt,interpolation="nearest")
    axs[1].set_title("Network input")
    
    outs = model.predict(np.array([imt]))
    print(outs)
    predicted = np.argmax(outs)
    axs[2].bar(np.array(range(len(names)))-0.5, outs[0,:], 1, color="gray")
    axs[2].set_ylim([0,1])
    axs[2].set_xticks(range(len(names)))
    axs[2].set_xticklabels(names)
    axs[2].set_ylabel("probability")
    axs[2].set_xlabel("class")
    axs[2].set_title("Network output")
    fig.tight_layout()
    plt.show()
    fig.savefig("out_{:05d}_{}.png".format(i,("ok" if predicted==l else "ko")))
    print(outs)

interact(resultsShow, i=widgets.IntSlider(min=0,max=len(dataset_te)-1, step=1, value=0, continuous_update=False), data=fixed(dataset_te.sample(len(dataset_te))), model=fixed(model))


Moreover, we can also visualize the contents of the hidden layers of the NN

In [ ]:
from keras import backend as K

def plot_hidden_layers(imt,nmaps=4):
    fig = plt.figure(figsize=(15, 10))
    plt.title('Hidden layers', loc='center')
    plt.axis('off')
    layers = [model.layers[0].input] + [model.layers[i].output for i in [0,2,5,7,9]]
    for layeri,layer in enumerate(layers):
        get = K.function([model.layers[0].input], [layer])
        layeroutputs = get([imt[np.newaxis,:,:,:]])[0][0]
        for j in range(nmaps): # for each map
            sp2=fig.add_subplot(nmaps,len(layers),layeri+1+j*len(layers))
            sp2.axis('off')
            if (layeroutputs.shape[2]>j):
                vmin,vmax = ((0,1) if layeri == 0 else (-0.3,+0.3))
                sp2.imshow(layeroutputs[:,:,j],
                           cmap="gray",
                           interpolation="nearest")
    fig.tight_layout()

imt=transform(dataset_te["image"].iloc[1],patchsize)
plot_hidden_layers(imt)
